In [6]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [7]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [4]:
1-0.0016*500

0.19999999999999996

In [5]:


gamma=0.99
epsilon_decay = 0.0015
learning_rate =0.0005
max_e=1
min_e=0
replay_batch_size=100
iterations=1500 

In [8]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = max_e-epsilon_decay*episode
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=5000:
        memory = memory[-5000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -165.89  rolling reward:  -165.89  Steps:  86  epsilon:  1.00
episode:  1  episode reward:  15.71  rolling reward:  -75.09  Steps:  138  epsilon:  1.00
episode:  2  episode reward:  -106.81  rolling reward:  -85.66  Steps:  109  epsilon:  1.00
episode:  3  episode reward:  -299.93  rolling reward:  -139.23  Steps:  100  epsilon:  1.00
episode:  4  episode reward:  -122.02  rolling reward:  -135.79  Steps:  100  epsilon:  0.99
episode:  5  episode reward:  -140.92  rolling reward:  -136.64  Steps:  92  epsilon:  0.99
episode:  6  episode reward:  -99.78  rolling reward:  -131.38  Steps:  59  epsilon:  0.99
episode:  7  episode reward:  -107.20  rolling reward:  -128.35  Steps:  76  epsilon:  0.99
episode:  8  episode reward:  -333.73  rolling reward:  -151.17  Steps:  91  epsilon:  0.99
episode:  9  episode reward:  -315.05  rolling reward:  -167.56  Steps:  81  epsilon:  0.99
episode:  10  episode reward:  -252.24  rolling reward:  -175.26  Steps:  78  eps

episode:  89  episode reward:  -131.95  rolling reward:  -153.37  Steps:  91  epsilon:  0.87
episode:  90  episode reward:  -213.48  rolling reward:  -154.03  Steps:  118  epsilon:  0.86
episode:  91  episode reward:  -154.36  rolling reward:  -154.03  Steps:  98  epsilon:  0.86
episode:  92  episode reward:  -32.11  rolling reward:  -152.72  Steps:  156  epsilon:  0.86
episode:  93  episode reward:  -153.13  rolling reward:  -152.73  Steps:  136  epsilon:  0.86
episode:  94  episode reward:  -97.22  rolling reward:  -152.14  Steps:  120  epsilon:  0.86
episode:  95  episode reward:  -148.96  rolling reward:  -152.11  Steps:  98  epsilon:  0.86
episode:  96  episode reward:  -105.27  rolling reward:  -151.63  Steps:  84  epsilon:  0.86
episode:  97  episode reward:  -192.90  rolling reward:  -152.05  Steps:  143  epsilon:  0.85
episode:  98  episode reward:  -155.52  rolling reward:  -152.08  Steps:  140  epsilon:  0.85
episode:  99  episode reward:  -76.71  rolling reward:  -151.33  S

episode:  177  episode reward:  -51.51  rolling reward:  -121.91  Steps:  97  epsilon:  0.73
episode:  178  episode reward:  -144.28  rolling reward:  -121.33  Steps:  84  epsilon:  0.73
episode:  179  episode reward:  -155.41  rolling reward:  -121.38  Steps:  124  epsilon:  0.73
episode:  180  episode reward:  -91.56  rolling reward:  -121.42  Steps:  86  epsilon:  0.73
episode:  181  episode reward:  -25.69  rolling reward:  -120.81  Steps:  156  epsilon:  0.73
episode:  182  episode reward:  -106.46  rolling reward:  -120.81  Steps:  102  epsilon:  0.73
episode:  183  episode reward:  -311.73  rolling reward:  -122.63  Steps:  100  epsilon:  0.73
episode:  184  episode reward:  -122.07  rolling reward:  -123.22  Steps:  97  epsilon:  0.72
episode:  185  episode reward:  -109.32  rolling reward:  -121.26  Steps:  124  epsilon:  0.72
episode:  186  episode reward:  -244.60  rolling reward:  -122.31  Steps:  129  epsilon:  0.72
episode:  187  episode reward:  -131.45  rolling reward: 

episode:  265  episode reward:  -84.51  rolling reward:  -123.36  Steps:  150  epsilon:  0.60
episode:  266  episode reward:  -139.14  rolling reward:  -123.97  Steps:  147  epsilon:  0.60
episode:  267  episode reward:  -205.43  rolling reward:  -123.88  Steps:  101  epsilon:  0.60
episode:  268  episode reward:  -44.05  rolling reward:  -123.20  Steps:  171  epsilon:  0.60
episode:  269  episode reward:  -69.86  rolling reward:  -120.27  Steps:  149  epsilon:  0.60
episode:  270  episode reward:  -105.59  rolling reward:  -119.09  Steps:  166  epsilon:  0.59
episode:  271  episode reward:  -71.66  rolling reward:  -118.57  Steps:  102  epsilon:  0.59
episode:  272  episode reward:  -51.24  rolling reward:  -115.17  Steps:  120  epsilon:  0.59
episode:  273  episode reward:  -67.58  rolling reward:  -112.18  Steps:  160  epsilon:  0.59
episode:  274  episode reward:  -73.13  rolling reward:  -112.46  Steps:  129  epsilon:  0.59
episode:  275  episode reward:  -82.39  rolling reward:  

episode:  353  episode reward:  104.19  rolling reward:  -80.77  Steps:  1000  epsilon:  0.47
episode:  354  episode reward:  -172.56  rolling reward:  -81.48  Steps:  527  epsilon:  0.47
episode:  355  episode reward:  -59.55  rolling reward:  -80.79  Steps:  186  epsilon:  0.47
episode:  356  episode reward:  145.21  rolling reward:  -76.51  Steps:  1000  epsilon:  0.47
episode:  357  episode reward:  -29.06  rolling reward:  -76.24  Steps:  147  epsilon:  0.46
episode:  358  episode reward:  -113.66  rolling reward:  -75.61  Steps:  232  epsilon:  0.46
episode:  359  episode reward:  -91.39  rolling reward:  -74.91  Steps:  138  epsilon:  0.46
episode:  360  episode reward:  -55.33  rolling reward:  -75.03  Steps:  140  epsilon:  0.46
episode:  361  episode reward:  -8.20  rolling reward:  -74.53  Steps:  169  epsilon:  0.46
episode:  362  episode reward:  140.80  rolling reward:  -73.01  Steps:  1000  epsilon:  0.46
episode:  363  episode reward:  -75.94  rolling reward:  -73.17  S

...

episode:  530  episode reward:  94.34  rolling reward:  -45.32  Steps:  1000  epsilon:  0.20
episode:  531  episode reward:  55.55  rolling reward:  -44.36  Steps:  1000  epsilon:  0.20
episode:  532  episode reward:  0.36  rolling reward:  -44.63  Steps:  201  epsilon:  0.20
episode:  533  episode reward:  -38.18  rolling reward:  -45.06  Steps:  147  epsilon:  0.20
episode:  534  episode reward:  -35.67  rolling reward:  -43.76  Steps:  328  epsilon:  0.20
episode:  535  episode reward:  -135.94  rolling reward:  -44.35  Steps:  188  epsilon:  0.20
episode:  536  episode reward:  -177.04  rolling reward:  -44.99  Steps:  773  epsilon:  0.20
episode:  537  episode reward:  -118.74  rolling reward:  -45.75  Steps:  1000  epsilon:  0.19
episode:  538  episode reward:  -215.47  rolling reward:  -48.07  Steps:  957  epsilon:  0.19
episode:  539  episode reward:  -28.29  rolling reward:  -49.21  Steps:  1000  epsilon:  0.19
episode:  540  episode reward:  -204.38  rolling reward:  -52.10  

episode:  618  episode reward:  -319.84  rolling reward:  -65.08  Steps:  341  epsilon:  0.07
episode:  619  episode reward:  -147.89  rolling reward:  -67.25  Steps:  149  epsilon:  0.07
episode:  620  episode reward:  -59.71  rolling reward:  -69.71  Steps:  1000  epsilon:  0.07
episode:  621  episode reward:  -195.51  rolling reward:  -73.72  Steps:  261  epsilon:  0.07
episode:  622  episode reward:  -475.41  rolling reward:  -78.55  Steps:  821  epsilon:  0.07
episode:  623  episode reward:  -196.96  rolling reward:  -80.56  Steps:  271  epsilon:  0.07
episode:  624  episode reward:  -23.98  rolling reward:  -81.26  Steps:  1000  epsilon:  0.06
episode:  625  episode reward:  -77.37  rolling reward:  -82.83  Steps:  1000  epsilon:  0.06
episode:  626  episode reward:  -65.67  rolling reward:  -84.52  Steps:  1000  epsilon:  0.06
episode:  627  episode reward:  -53.83  rolling reward:  -86.46  Steps:  1000  epsilon:  0.06
episode:  628  episode reward:  -85.54  rolling reward:  -87

episode:  704  episode reward:  -119.89  rolling reward:  -120.64  Steps:  1000  epsilon:  -0.06
episode:  705  episode reward:  -235.37  rolling reward:  -121.68  Steps:  166  epsilon:  -0.06
episode:  706  episode reward:  -204.97  rolling reward:  -124.00  Steps:  139  epsilon:  -0.06
episode:  707  episode reward:  -190.15  rolling reward:  -125.00  Steps:  653  epsilon:  -0.06
episode:  708  episode reward:  -157.05  rolling reward:  -126.10  Steps:  1000  epsilon:  -0.06
episode:  709  episode reward:  -79.04  rolling reward:  -126.08  Steps:  1000  epsilon:  -0.06
episode:  710  episode reward:  -179.16  rolling reward:  -126.25  Steps:  1000  epsilon:  -0.06
episode:  711  episode reward:  -79.50  rolling reward:  -124.83  Steps:  1000  epsilon:  -0.07
episode:  712  episode reward:  -118.68  rolling reward:  -124.63  Steps:  111  epsilon:  -0.07
episode:  713  episode reward:  -330.12  rolling reward:  -127.46  Steps:  684  epsilon:  -0.07
episode:  714  episode reward:  -305.

episode:  789  episode reward:  -420.50  rolling reward:  -169.25  Steps:  410  epsilon:  -0.18
episode:  790  episode reward:  -368.91  rolling reward:  -169.91  Steps:  287  epsilon:  -0.19
episode:  791  episode reward:  -279.25  rolling reward:  -172.31  Steps:  604  epsilon:  -0.19
episode:  792  episode reward:  -405.77  rolling reward:  -175.36  Steps:  713  epsilon:  -0.19
episode:  793  episode reward:  -208.98  rolling reward:  -175.58  Steps:  1000  epsilon:  -0.19
episode:  794  episode reward:  -406.08  rolling reward:  -178.19  Steps:  389  epsilon:  -0.19
episode:  795  episode reward:  -444.71  rolling reward:  -181.38  Steps:  688  epsilon:  -0.19
episode:  796  episode reward:  -524.61  rolling reward:  -184.70  Steps:  1000  epsilon:  -0.19
episode:  797  episode reward:  -146.60  rolling reward:  -185.08  Steps:  1000  epsilon:  -0.20
episode:  798  episode reward:  -175.23  rolling reward:  -185.77  Steps:  1000  epsilon:  -0.20
episode:  799  episode reward:  -149

KeyboardInterrupt: 

In [ ]:
online_model.save_weights('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 